In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import itertools 
import pickle
from ast import literal_eval
import sys
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from collections import Counter
from math import sqrt

## ConSurv-XGB - Jaccard similiarity

In [2]:
seeds = [999, 7, 42, 1995, 1303, 2405, 1996, 200, 0, 777]
tree_idx_ls = []
tree_idx_frac_ls = []
seeds_ls = []
nconditions_ls = []
topn = 100
fontsize = 14

jidx_xgb_ls = []

for idx in range(len(seeds)):
    seed1 = seeds[idx]
    # Read concept weights from model
    with open('./../models/XGBMLP/concept_weights_seed'+str(seed1)+'.pkl','rb') as f:
        dict_ = pickle.load(f)
    f.close()
    concepts_weights_df = pd.DataFrame(dict_)
    concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
    concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
    concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.iloc[:topn]
    cls1 = concepts_weights_df['feature_groups_idx'].to_list()  
    
    for seed2 in seeds[idx+1:]:
        jidx_ls_ = []
        # Read concept weights from model
        with open('./../models/XGBMLP/concept_weights_seed'+str(seed2)+'.pkl','rb') as f:
            dict_ = pickle.load(f)
        f.close()
        concepts_weights_df = pd.DataFrame(dict_)
        concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
        concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
        concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.iloc[:topn]
        cls2 = concepts_weights_df['feature_groups_idx'].to_list()
        
        jidx_ls_ = [float(len(set(cls1[i]).intersection(set(cls2[j]))))/float(len(set(cls1[i]).union(set(cls2[j]))))
                    for i in range(topn) for j in range(i+1,topn)]
        
        jidx_xgb_ls = jidx_xgb_ls + [jidx_ls_]
jidx_xgb_ls

with open('./../results/closeness/jidx_xgb_ls.pkl','wb') as f:
    pickle.dump(jidx_xgb_ls,f)
f.close()

In [3]:
len(jidx_xgb_ls), len(jidx_xgb_ls[0])

(45, 4950)

## ConSurv-XGB - Cosine similiarity

In [4]:
seeds = [999, 7, 42, 1995, 1303, 2405, 1996, 200, 0, 777]
tree_idx_ls = []
tree_idx_frac_ls = []
seeds_ls = []
nconditions_ls = []
topn = 100
fontsize = 14

cidx_xgb_ls = []

for idx in range(len(seeds)):
    seed1 = seeds[idx]
    
    # Read concept weights from model
    with open('./../models/XGBMLP/concept_weights_seed'+str(seed1)+'.pkl','rb') as f:
        dict_ = pickle.load(f)
    f.close()
    concepts_weights_df = pd.DataFrame(dict_)
    concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
    concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
    concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.iloc[:topn]
    cls1 = concepts_weights_df['feature_groups_idx'].to_list()  
    
    for seed2 in seeds[idx+1:]:
        cidx_ls_ = []

        # Read concept weights from model
        with open('./../models/XGBMLP/concept_weights_seed'+str(seed2)+'.pkl','rb') as f:
            dict_ = pickle.load(f)
        f.close()
        concepts_weights_df = pd.DataFrame(dict_)
        concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
        concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
        concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.iloc[:topn]
        cls2 = concepts_weights_df['feature_groups_idx'].to_list()

        for i in range(topn):
            for j in range(i+1,topn):
                cls = list(set(cls1[i]+cls2[j]))
                
                # Create binary vectors for both sets
                cls1_ = [1 if item in cls1[i] else 0 for item in cls]
                cls2_ = [1 if item in cls2[j] else 0 for item in cls]
                
                # Compute dot product of the two vectors
                dot_product = sum(a*b for a,b in zip(cls1_, cls2_))
                
                # Compute magnitudes of the vectors
                magnitude_a = sqrt(sum(a**2 for a in cls1_))
                magnitude_b = sqrt(sum(b**2 for b in cls2_))
                
                # Avoid division by zero
                if magnitude_a == 0 or magnitude_b == 0:
                    cidx_ls_ = cidx_ls_ + [0]

                else: # Compute cosine similarity
                    cidx_ls_ = cidx_ls_ + [dot_product / (magnitude_a * magnitude_b)]
                        
        cidx_xgb_ls = cidx_xgb_ls + [cidx_ls_]
cidx_xgb_ls

with open('./../results/closeness/cidx_xgb_ls.pkl','wb') as f:
    pickle.dump(cidx_xgb_ls,f)
f.close()

In [5]:
len(cidx_xgb_ls), len(cidx_xgb_ls[0])

(45, 4950)

## ConSurv-Rule - Jaccard similiarity

In [2]:
seeds = [999, 7, 42, 1995, 1303, 2405, 1996, 200, 0, 777]
tree_idx_ls = []
tree_idx_frac_ls = []
seeds_ls = []
nconditions_ls = []
topn = 25
fontsize = 14

jidx_rule_ls = []

for idx in range(len(seeds)):
    seed1 = seeds[idx]
    # Read concept weights from model
    with open('./../models/RuleMLP/ntopfeatures/catrulekit_3class_1hot_concept_weights_top25seed'+str(seed1)+'.pkl','rb') as f:
        dict_ = pickle.load(f)
    f.close()
    concepts_weights_df = pd.DataFrame(dict_)
    concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
    concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
    concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.iloc[:topn]
    cls1 = concepts_weights_df['feature_groups_idx'].to_list()  
    
    for seed2 in seeds[idx+1:]:
        jidx_ls_ = []
        # Read concept weights from model
        with open('./../models/RuleMLP/ntopfeatures/catrulekit_3class_1hot_concept_weights_top25seed'+str(seed2)+'.pkl','rb') as f:
            dict_ = pickle.load(f)
        f.close()
        concepts_weights_df = pd.DataFrame(dict_)
        concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
        concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
        concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.iloc[:topn]
        cls2 = concepts_weights_df['feature_groups_idx'].to_list()
        
        jidx_ls_ = [float(len(set(cls1[i]).intersection(set(cls2[j]))))/float(len(set(cls1[i]).union(set(cls2[j]))))
                    for i in range(topn) for j in range(i+1,topn)]
        
        jidx_rule_ls = jidx_rule_ls + [jidx_ls_]
jidx_rule_ls

with open('./../results/closeness/jidx_rule_ls.pkl','wb') as f:
    pickle.dump(jidx_rule_ls,f)
f.close()

In [3]:
len(jidx_rule_ls), len(jidx_rule_ls[0])

(45, 300)

## ConSurv-Rule - Cosine similiarity

In [4]:
seeds = [999, 7, 42, 1995, 1303, 2405, 1996, 200, 0, 777]
tree_idx_ls = []
tree_idx_frac_ls = []
seeds_ls = []
nconditions_ls = []
topn = 25
fontsize = 14

cidx_rule_ls = []

for idx in range(len(seeds)):
    seed1 = seeds[idx]
    
    # Read concept weights from model
    with open('./../models/RuleMLP/ntopfeatures/catrulekit_3class_1hot_concept_weights_top25seed'+str(seed1)+'.pkl','rb') as f:
        dict_ = pickle.load(f)
    f.close()
    concepts_weights_df = pd.DataFrame(dict_)
    concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
    concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
    concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
    concepts_weights_df = concepts_weights_df.iloc[:topn]
    cls1 = concepts_weights_df['feature_groups_idx'].to_list()  
    
    for seed2 in seeds[idx+1:]:
        cidx_ls_ = []

        # Read concept weights from model
        with open('./../models/RuleMLP/ntopfeatures/catrulekit_3class_1hot_concept_weights_top25seed'+str(seed1)+'.pkl','rb') as f:
            dict_ = pickle.load(f)
        f.close()
        concepts_weights_df = pd.DataFrame(dict_)
        concepts_weights_df['abs_weights'] = [np.abs(i) for i in concepts_weights_df['weights']]
        concepts_weights_df['concepts'] = ['concept'+str(i) for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.sort_values('abs_weights',ascending=False)
        concepts_weights_df['ranking'] = [i+1 for i in range(len(concepts_weights_df))]
        concepts_weights_df = concepts_weights_df.iloc[:topn]
        cls2 = concepts_weights_df['feature_groups_idx'].to_list()

        for i in range(topn):
            for j in range(i+1,topn):
                cls = list(set(cls1[i]+cls2[j]))
                
                # Create binary vectors for both sets
                cls1_ = [1 if item in cls1[i] else 0 for item in cls]
                cls2_ = [1 if item in cls2[j] else 0 for item in cls]
                
                # Compute dot product of the two vectors
                dot_product = sum(a*b for a,b in zip(cls1_, cls2_))
                
                # Compute magnitudes of the vectors
                magnitude_a = sqrt(sum(a**2 for a in cls1_))
                magnitude_b = sqrt(sum(b**2 for b in cls2_))
                
                # Avoid division by zero
                if magnitude_a == 0 or magnitude_b == 0:
                    cidx_ls_ = cidx_ls_ + [0]

                else: # Compute cosine similarity
                    cidx_ls_ = cidx_ls_ + [dot_product / (magnitude_a * magnitude_b)]
                        
        cidx_rule_ls = cidx_rule_ls + [cidx_ls_]
cidx_rule_ls

with open('./../results/closeness/cidx_rule_ls.pkl','wb') as f:
    pickle.dump(cidx_rule_ls,f)
f.close()

In [5]:
len(jidx_rule_ls), len(jidx_rule_ls[0])

(45, 300)